In [1]:
import h5py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from pyXSteam.XSteam import XSteam
#from scipy.interpolate import interp1d
#from scipy.sparse import coo_matrix


In [86]:
def sigmaZeros(sigTtab, sig0tab, aDen, SigEscape):
    # Number of energy groups
    ng = 421

    # Define number of isotopes in the mixture
    nIso = len(aDen)

    # first guess for sigma-zeros is 1e10 (infinite dilution)
    sig0 = np.ones((nIso, ng)) * 1e10

    # Loop over energy group
    for ig in range(ng):
        # Error to control sigma-zero iterations
        err = 1e10
        nIter = 0

        # sigma-sero iterations until the error is below selected tolerance (1e-6)
        while err > 1e-6:
            # Loop over isotopes
            for iIso in range(nIso):
                # Find cross section for the current sigma-zero by interpolating
                # in the table
                if np.count_nonzero(sig0tab[iIso]) == 1:
                    sigT = sigTtab[iIso][0, ig]
                else:
                    log10sig0 = np.minimum(10, np.maximum(0, np.log10(sig0[iIso, ig])))
                    # TODO: fix the line below!!!
                    sigT = sp.interpolate.interp1d(np.log10(sig0tab[iIso]), sigTtab[iIso][:, ig], kind='linear')(log10sig0)
                sigT = np.asscalar(sigT)
                sigTtab[iIso][np.isnan(sigTtab[iIso])] = 0

            err = 0
            # Loop over isotopes
            for iIso in range(nIso):
                # Find the total macroscopic cross section for the mixture of
                # the background isotopes
                sum = 0
                # Loop over background isotopes
                for jIso in range(nIso):
                    if jIso != iIso:
                        sum += sigTtab[jIso][ig] * aDen[jIso]

                tmp = (SigEscape + sum) / aDen[iIso]
                err += (1 - tmp / sig0[iIso, ig]) ** 2
                sig0[iIso, ig] = tmp

            err = np.sqrt(err)
            nIter += 1
            if nIter > 100:
                print('Error: too many sigma-zero iterations.')
                return

    return sig0

In [145]:
# number of energy groups
H2OU_ng = 421

# Path to microscopic cross section data:
micro_XS_path = '../01.Micro_Python'

# Call the functions for H2O and B isotopes and store the data in the
# structures. As an example it is done below for temperature of 294K,
# pressure of 7 MPa and boron concentration of 760 ppm.
# Change when other parameters needed.
# Open the HDF5 files
hdf5_H01  = h5py.File(micro_XS_path + '/micro_H_001__294K.h5', 'r')
hdf5_O16  = h5py.File(micro_XS_path + '/micro_O_016__294K.h5', 'r')
hdf5_U235 = h5py.File(micro_XS_path + '/micro_U_235__294K.h5', 'r')

# For looking inside the HDF5 file groups
element = 'H01'
group = 'sigT_G'
subgroup = 'sigT(0,:)'
print(f'Metadata for {element}:', list(eval(f'hdf5_{element}').attrs.keys()))
baseItems = list(eval(f'hdf5_{element}').items())
print(f'Base Items in {element}: ',baseItems)
#eg_G_H01 = hdf5_H01.get('en_G')
groupItems = list(hdf5_H01.get(group).items())
print(f'Items in {group}_{element}:', groupItems)
subgroupData = np.array(hdf5_H01.get(group).get(subgroup))
print(f'NumPy data in {subgroup}:\n',subgroupData[0:5])

H2OU_temp = 294  # K
H2OU_p = 0.1  # MPa
H2OU_Uconc = 1000e-6  # 1e-6 = 1 ppm
#H2OU_eg = H01['eg']
H2OU_eg = np.array(hdf5_H01.get('en_G').get('eg'))   # From the group 'en_G' get subgroup containing the data named 'eg'

# Get the atomic weight from the metadata
H01_aw  = hdf5_H01.attrs.get('atomic_weight')
O16_aw  = hdf5_O16.attrs.get('atomic_weight')
U235_aw = hdf5_U235.attrs.get('atomic_weight')
print('\natomic_weight of H01: ', H01_aw,'\ndata type:', type(H01_aw))

# Mass of one "average" H2OU molecule in atomic unit mass [a.u.m.]:
H2OU_aw = 2 * H01_aw + O16_aw + H2OU_Uconc * U235_aw

# Path to steam-water properties:
#XSteam_path = '../00.pyXSteam'

# The function returns water density at specified pressure (MPa) and
# temperature (C):
steamTable = XSteam(XSteam.UNIT_SYSTEM_MKS)
density = steamTable.rho_pt(H2OU_p*10, H2OU_temp-273)

# The water density:
H2OU_den = density*1e-3  # [g/cm3]
rho = H2OU_den*1.0e-24  # [g/(barn*cm)]
rho = rho / 1.660538e-24  # [(a.u.m.)/(barn*cm)]
rho = rho / H2OU_aw  # [number of H2O molecules/(barn*cm)]

# The names of fissionable isotopes and oxygen
H2OU_isoName = ['H01', 'O16', 'U235']

# The number densities of isotopes:
H2OU_numDen = [2*rho, rho, rho*H2OU_Uconc]

# I have no idea if this is working correctly, but the
# idea is that if a group, let's say sigT_G, has multible
# subgroups named 'sigT(0,:)', 'sigT(1,:)', ..., 'sigT(9,:)'
# then this function should combine all of them into
# one single NumPy array 
def get_subg_data_as_array(group):
    subgroups_data = []

    def get_data(name, obj):
        if isinstance(obj, h5py.Dataset):
            subgroup_data = np.array(obj)
            subgroups_data.append(subgroup_data)

    group.visititems(get_data)
    return np.array(subgroups_data) # The output is np.array([[data]]) 
                                    # That is useful for np.concatenate()
# end of function                       

# Testing
#sigT_G = hdf5_U235.get('sigT_G')
#sigT_data = get_subg_data_as_array(sigT_G)
#print(sigT_G, sigT_data)

# Get total micrscopic cross-section data 
sigT_H01 = get_subg_data_as_array(hdf5_H01.get('sigT_G'))
sigT_O16 = get_subg_data_as_array(hdf5_O16.get('sigT_G'))
sigT_U235 = get_subg_data_as_array(hdf5_U235.get('sigT_G'))
# Prepare for sigma-zero iterations:
# Concatenate the arrays along the first axis to get a 3D array
sigTtab = np.concatenate([sigT_H01, sigT_O16, sigT_U235], axis=0)

# wtf is sigma zero?
sig0_H01  = np.array(hdf5_H01.get( 'sig0_G').get('sig0'))
sig0_O16  = np.array(hdf5_O16.get( 'sig0_G').get('sig0'))
sig0_U235 = np.array(hdf5_U235.get('sig0_G').get('sig0'))
sig0tab = np.concatenate([sig0_H01, sig0_O16, sig0_U235], axis=0)

# NOTE: currently concatenate puts all the data in single vector
# but in the Matlab code, all data vectors are added separately
# and every cell in the resulting 3D/2D matrix have different lengths
# Determine the length of each sig0_* variable
sig0_sizes = [len(sig0_H01), len(sig0_O16), len(sig0_U235)]
isotope = ["H01", "O16", "U235"]
# Create a 2D array where the number of columns is determined by the length of the sig0_* variables
sig0tab = np.empty((3, max(sig0_sizes)))    # (3,10)
col_start = 0
for i, size in enumerate(sig0_sizes):
    print("i =", i, "size =", size)
    sig0tab[i, col_start:col_start+size] = eval(f'sig0_{isotope[i]}')
    #col_start += size


aDen = H2OU_numDen

# SigEscape -- escape cross section, for simple convex objects (such as
# plates, spheres, or cylinders) is given by S/(4V), where V and S are the
# volume and surface area of the object, respectively
SigEscape = 0

print('Sigma-zero iterations. ')
# TODO: figure out interpolation in this function
H2OU_sig0 = sigmaZeros(sigTtab, sig0tab, aDen, SigEscape)
print('Done.\n')

# Close the HDF5 files
hdf5_H01.close()
hdf5_O16.close()
hdf5_U235.close()


Metadata for H01: ['atomic_weight', 'description', 'ng', 'temperature']
Base Items in H01:  [('chi_G', <HDF5 group "/chi_G" (1 members)>), ('en_G', <HDF5 group "/en_G" (1 members)>), ('nubar_G', <HDF5 group "/nubar_G" (1 members)>), ('sig0_G', <HDF5 group "/sig0_G" (2 members)>), ('sig2_G', <HDF5 group "/sig2_G" (1 members)>), ('sigC_G', <HDF5 group "/sigC_G" (1 members)>), ('sigE_G', <HDF5 group "/sigE_G" (1 members)>), ('sigF_G', <HDF5 group "/sigF_G" (1 members)>), ('sigS_G', <HDF5 group "/sigS_G" (5 members)>), ('sigT_G', <HDF5 group "/sigT_G" (1 members)>)]
Items in sigT_G_H01: [('sigT(0,:)', <HDF5 dataset "sigT(0,:)": shape (421,), type "<f8">)]
NumPy data in sigT(0,:):
 [355.20442296 170.92080275 133.09867696 120.43656698 113.23030272]

atomic_weight of H01:  1.007825000724447 
data type: <class 'numpy.float64'>
i = 0 size = 1
i = 1 size = 6
i = 2 size = 10
Sigma-zero iterations. 


/tmp/ipykernel_218080/3064007386.py:27: RuntimeWarning: divide by zero encountered in log10
  sigT = sp.interpolate.interp1d(np.log10(sig0tab[iIso]), sigTtab[iIso][:, ig], kind='linear')(log10sig0)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [144]:
np.count_nonzero(sig0tab[2])

10

In [80]:
hdf5_U235 = h5py.File(micro_XS_path + '/micro_U_235__294K.h5', 'r')

def get_subgroup_data_as_array(group):
    subgroups_data = []

    def get_data(name, obj):
        if isinstance(obj, h5py.Dataset):
            subgroup_data = np.array(obj)
            subgroups_data.append(subgroup_data)

    group.visititems(get_data)
    return np.array(subgroups_data)[0]

sigT_G_test = hdf5_U235.get('sigT_G')
sigT_data = get_subgroup_data_as_array(sigT_G_test)

print(sigT_G_test)
print(sigT_data)


<HDF5 group "/sigT_G" (10 members)>
[15522.986351    6729.12259768  4740.19746511  4009.70562995
  3577.66532099  3228.79369954  2834.27291974  2499.00442217
  2259.13333722  1999.19301603  1760.65658155  1487.51059223
  1252.48849331   870.76454325   684.12648427   602.04544033
   522.82148174   465.85815532   422.42401193   388.11353298
   360.22670954   337.25126649   306.81336106   275.11215392
   255.49179303   246.18434613   246.87312252   256.65779245
   269.45728865   272.04373311   255.83800503   228.22000511
   200.53275628   177.90018984   154.24787579   133.25518776
   119.91305784   110.78782182   105.59795242   102.83943934
    99.52275532    96.21857214    94.19516546    93.45628155
    94.33707817    96.34535879    98.68524546   102.09079328
   106.91500907   111.36001065   114.45036201   117.96176774
   121.94928587   126.46041533   131.51000771   137.09219259
   143.12319159   149.40983328   155.67998695   161.4850484
   166.29701815   169.36757841   170.03357052   16

In [6]:
def interpSigS(jLgn, s, Sig0):
    # number of energy groups
    ng = 421

    # number of sigma-zeros
    nSig0 = len(s['sig0'])
    if nSig0 == 1:
        sigS = s['sigS'][jLgn][0]
    else:
        tmp1 = np.zeros((nSig0, ng*ng))
        for iSig0 in range(nSig0):
            ifrom, ito, tmp1[iSig0,:] = sp.sparse.find(s['sigS'][jLgn][iSig0])
        
        # number of non-zeros in a scattering matrix
        nNonZeros = tmp1.shape[1]
        tmp2 = np.zeros(nNonZeros)
        for i in range(nNonZeros):
            log10sig0 = min(10, max(0, np.log10(Sig0[ifrom[i]])))
            tmp2[i] = sp.interpolate.interp1d(np.log10(s['sig0']), tmp1[:,i], kind='linear')(log10sig0)
        
        sigS = sp.sparse.coo_matrix((tmp2, (ifrom, ito)), shape=(ng, ng)).toarray()

    return sigS


In [23]:
"""x = np.arange(0, 10)
y = np.exp(-x/3.0)
f = sp.interpolate.interp1d(x, y)
plt.plot(x, y, 'o', x, y, '-')
plt.show()"""

"x = np.arange(0, 10)\ny = np.exp(-x/3.0)\nf = sp.interpolate.interp1d(x, y)\nplt.plot(x, y, 'o', x, y, '-')\nplt.show()"

In [24]:
"""xnew = np.arange(0, 9, 0.1)
ynew = f(xnew)   # use interpolation function returned by `interp1d`
plt.plot(x, y, 'o', xnew, ynew, '-')
plt.show()"""

"xnew = np.arange(0, 9, 0.1)\nynew = f(xnew)   # use interpolation function returned by `interp1d`\nplt.plot(x, y, 'o', xnew, ynew, '-')\nplt.show()"

In [ ]:
# number of energy groups
H2OU_ng = 421

# Path to microscopic cross section data:
micro_XS_path = '../01.Micro_Python'

# Call the functions for H2O and B isotopes and store the data in the
# structures. As an example it is done below for temperature of 294K,
# pressure of 7 MPa and boron concentration of 760 ppm.
# Change when other parameters needed.
# Open the HDF5 files
hdf5_H01 = h5py.File('micro_H_001__294K.h5', 'r')
hdf5_O16 = h5py.File('micro_O_016__294K.h5', 'r')
hdf5_U235 = h5py.File('micro_U_235__294K.h5', 'r')

# Read the datasets from the HDF5 files

#H01 = hdf5_H01['dataset_name'][()]
#O16 = hdf5_O16['dataset_name'][()]
#U235 = hdf5_U235['dataset_name'][()]

# Close the HDF5 files
hdf5_H01.close()
hdf5_O16.close()
hdf5_U235.close()

# Something similar maybe?
#H01 = np.load(micro_XS_path + '/micro_H_001__294K.npy')
#O16 = np.load(micro_XS_path + '/micro_O_016__294K.npy')
#U235 = np.load(micro_XS_path + '/micro_U_235__294K.npy')
"""
H2OU_temp = 294  # K
H2OU_p = 0.1  # MPa
H2OU_Uconc = 1000e-6  # 1e-6 = 1 ppm
H2OU_eg = H01['eg']

# Mass of one "average" H2OU molecule in atomic unit mass [a.u.m.]:
H2OU_aw = 2 * H01['aw'] + O16['aw'] + H2OU_Uconc * U235['aw']

# Path to steam-water properties:
XSteam_path = '../00.XSteam'

# The function returns water density at specified pressure (MPa) and
# temperature (C):
import sys
sys.path.append(XSteam_path)
from XSteam import XSteam
steamTable = XSteam(XSteam_path)
density = steamTable.rho_pt(H2OU_p*10, H2OU_temp-273)

# The water density:
H2OU_den = density*1e-3  # [g/cm3]
rho = H2OU_den*1.0e-24  # [g/(barn*cm)]
rho = rho / 1.660538e-24  # [(a.u.m.)/(barn*cm)]
rho = rho / H2OU_aw  # [number of H2O molecules/(barn*cm)]

# The names of fissionable isotopes and oxygen
H2OU_isoName = ['H01', 'O16', 'U235']

# The number densities of isotopes:
H2OU_numDen = [2*rho, rho, rho*H2OU_Uconc]

# Prepare for sigma-zero iterations:
sigTtab = [H01['sigT'], O16['sigT'], U235['sigT']]
sig0tab = [H01['sig0'], O16['sig0'], U235['sig0']]
aDen = H2OU_numDen

# SigEscape -- escape cross section, for simple convex objects (such as
# plates, spheres, or cylinders) is given by S/(4V), where V and S are the
# volume and surface area of the object, respectively
SigEscape = 0

print('Sigma-zero iterations. ')
H2OU_sig0 = sigmaZeros(sigTtab, sig0tab, aDen, SigEscape)
print('Done.\n')
"""